In [4]:
import re
import json

receipt_text = """ДУБЛИКАТ
Филиал ТОО EUROPHARMA Астана
БИН 080841000761
НДС Серия 58001
 № 0014377
Касса 300-189
Смена 10
Порядковый номер чека №64
Чек №2331180266
Кассир Аптека 17-1
ПРОДАЖА
1.
Натрия хлорид 0,9%, 200 мл, фл
2,000 x 154,00
308,00
Стоимость
308,00
...
Банковская карта:
18 009,00
ИТОГО:
18 009,00
в т.ч. НДС 12%:
0,00
Фискальный признак:
2331180266
Время: 18.04.2019 11:13:58
г. Нур-султан,Казахстан, Мангилик Ел,19, нп-5
Оператор фискальных данных: АО"Казахтелеком"Для проверки чека зайдите на сайт:consumer.oofd.kz
ФИСКАЛЬНЫЙ ЧЕК
ФП
ИНК ОФД: 311559
Код ККМ КГД (РНМ): 620300145316
ЗНМ: SWK00034961
WEBKASSA.KZ"""

patterns = {
    "branch": r"Филиал ТОО (.+)",
    "bin": r"БИН (\d+)",
    "nds_series": r"НДС Серия (\d+)",
    "kassa": r"Касса (\S+)",
    "smena": r"Смена (\d+)",
    "sequence_number": r"Порядковый номер чека №(\d+)",
    "check_number": r"Чек №(\d+)",
    "kassir": r"Кассир (.+)",
    "items": r"(\d+)\.\n(.+?)\n(\d+,\d+) x (\d+,\d+)\n(\d+,\d+)"
}

receipt_data = {}

for key, pattern in patterns.items():
    if key == "items":
        matches = re.findall(pattern, receipt_text, re.DOTALL)
        receipt_data[key] = [
            {
                "id": int(match[0]),
                "name": match[1].strip(),
                "price": float(match[2].replace(',', '.')),
                "quantity": float(match[3].replace(',', '.')),
                "sum": float(match[4].replace(',', '.'))
            }
            for match in matches
        ]
    else:
        match = re.search(pattern, receipt_text)
        if match:
            receipt_data[key] = match.group(1).strip()

formatted_json = json.dumps(receipt_data, indent=4, ensure_ascii=False)
print(formatted_json)



{
    "branch": "EUROPHARMA Астана",
    "bin": "080841000761",
    "nds_series": "58001",
    "kassa": "300-189",
    "smena": "10",
    "sequence_number": "64",
    "check_number": "2331180266",
    "kassir": "Аптека 17-1",
    "items": [
        {
            "id": 1,
            "name": "Натрия хлорид 0,9%, 200 мл, фл",
            "price": 2.0,
            "quantity": 154.0,
            "sum": 308.0
        }
    ]
}
